# Análisis de datos de minst

In [1]:
import gzip, pickle
import math
import numpy as np
with gzip.open('mnist.pkl.gz','rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    train_set, val_set, test_set = u.load()
print(train_set[0].shape, train_set[1].shape)
print(val_set[0].shape, val_set[1].shape)
print(test_set[0].shape, test_set[1].shape)


(50000, 784) (50000,)
(10000, 784) (10000,)
(10000, 784) (10000,)


In [2]:
lista=list()
for i in range(train_set[0].shape[0]):
    if int(train_set[1][i])==1 or int(train_set[1][i])==0:
        lista.append((int(train_set[1][i]),train_set[0][i]))
        
#print(lista[0][1])


In [48]:
from numpy import linalg as la
def pi_x(x,i):
    #print(x)
    var=(1.0)/(1.0+math.exp(-1*np.dot(lista[i][1].T,x[:-1])-x[-1])+1e-10)
    #print(var)
    return var

def log_likelihood(beta):
    suma=0
    for i in range(len(lista)):
        beta=np.zeros(785,dtype=np.float32)
        suma += lista[i][0]*math.log(pi_x(beta,i))+(1-lista[i][0])*math.log(1-pi_x(beta,i))
    return suma

In [49]:
def der_pix(x,i):
    k=math.exp(-1*np.dot(lista[i][1].T,x[:-1])-x[-1])
    l=(1+math.exp(-1*np.dot(lista[i][1].T,x[:-1])-x[-1]))**2
    vec=np.zeros(785,dtype=np.float32)
    for j in range(784):
        vec[j]=lista[i][1][j]
    vec[784]=1
    return k/l*vec

def der_likelihood(beta):
    return -1*sum((lista[i][0]/pi_x(beta,i)-((1-lista[i][0])/(1-pi_x(beta,i))))*der_pix(beta,i) for i in range(len(lista)))

In [56]:
def backtracking_direction(x,a,p,c,dd,tf): #Veremos si el alpha satisface la condción de Armijo para el suficiente descenso
    alpha=a
    while tf(x+alpha*dd)> tf(x)+c*alpha*np.dot(dd.T,dd):
        alpha=p*alpha
    return alpha #Este es el alpha óptimo 

def Barzilai_Borwein_descenso(x0,maxitr,epsilon,grad,tf): #la función en este caso no se ocupa
    k=0
    g_k=grad(x0)
    g=-1*g_k
    g=g/la.norm(g)
    x_k=x0
    iterations=0
    while la.norm(g_k)>epsilon and iterations<maxitr:
        if k==0:
            a=backtracking_direction(x_k,0.2,0.5,0.5,g,tf)
            #print(a)
        else:
            a=np.dot((x_k-x_kant).T,(g_k-g_k_t))*(1/np.dot((g_k-g_k_t).T,(g_k-g_k_t)))
            #print(a)
        x_kant=x_k
        g_k_t=grad(x_kant)
        x_k=x_k+a*g
        g_k=grad(x_k)
        g=-1*g_k
        k=k+1
        iterations=iterations+1
        print(la.norm(g_k))
    return np.array([x_k,g_k])

def modified_backtracking_direction(x,C,a,p,c1,dd,tf): #Veremos si el alpha satisface la condción de Armijo para el suficiente descenso
    alpha=a
    while tf(x+alpha*dd) > C +c1*alpha*np.dot(dd.T,dd):
        alpha=p*alpha
    return alpha #Este es el alpha óptimo 

def Zhang_Hager(x0,maxitr,epsilon,nu,grad,tf):
    k=0
    g_k=grad(x0)
    g=-1*g_k
    iterations=0
    Q=1
    x_k=x0
    C=tf(x_k)
    trial=modified_backtracking_direction(x_k,C,1,0.5,0.5,g,tf)
    while la.norm(g_k)>epsilon and iterations<maxitr:
        a=modified_backtracking_direction(x_k,C,trial,0.5,0.5,g,tf)
        trial=a
        Q_ant=Q
        x_k=x_k+a*g
        g_k=grad(x_k)
        g=-1*g_k
        Q=Q_ant+1
        C=(nu*Q_ant*C+tf(x_k))/Q
        k=k+1
    return np.array([x_k,g_k])

In [57]:
puntoinic=np.zeros(785,dtype=np.float32)
#res=der_likelihood(puntoinic)
#print(la.norm(res))

In [58]:
res=Barzilai_Borwein_descenso(puntoinic,1000,1e-4,der_likelihood,log_likelihood)
print(res)

28319.805
13837.741
6700.988
4522.338
2002.9633
1087.6434
560.93787
309.40857
201.28621
169.50356
108.9115
95.683235
70.79222
38.132683
69.238716
36.035004
22.473394
21.797468
17.406712
11.781034
124.91674
8.690345
8.3056965
8.0655
7.36252
9.180305
10.177657
6.462254
6.411109
3.8745039
34.864105
30.410202
6.5293193
6.0855684
1.9037592
2.9216647
3.283677
1.4849443
1.4743469
1.3100882
1.2986461
2.1177537
4.694052
1.0171516
0.9989344
0.9369919
0.88631636
0.76298124
9.519317
0.7249214
0.68119174
0.59706026
0.5406373
0.9304023
0.5003852
0.49743488
0.4916673
0.19349962
3.400626
0.1790331
0.15359302
0.14148486
0.13056827
0.10866197
0.2545051
0.09874032
0.09674931
0.09619882
0.07514287
0.2522466
0.6353304
0.06674586
0.06457299
0.05481665
0.05314637
0.19639792
0.0437037
0.043515895
0.04017415
0.021067254
0.022730483
7.74343
0.021483207
0.021152006
0.01806619
0.016162837
0.013749187
0.014581236
0.016737008
0.009570984
0.009381795
0.008223379
0.00795134
0.023332145
0.007250246
0.007205819
0.00715

In [59]:
print(res)

[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 8.4588823e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 2.3479744e-05]]


In [62]:
puntomax=res[0]

In [75]:
test_list=list()
for i in range(test_set[0].shape[0]):
    if int(test_set[1][i])==1 or int(test_set[1][i])==0:
        test_list.append((int(test_set[1][i]),test_set[0][i]))

def pi_xtest(x,i):
    #print(x)
    var=(1.0)/(1.0+math.exp(-1*np.dot(test_list[i][1].T,x[:-1])-x[-1])+1e-10)
    #print(var)
    return var        

In [79]:
def indicadora_beta1_0(beta,i):
    if pi_xtest(beta,i)>0.5:
        res=1
    else: 
        res=0
    return res

def error_pred(beta):
    return (1/len(test_list))*sum(abs(indicadora_beta1_0(beta,i)-test_list[i][0]) for i in range(len(test_list)))
    

In [80]:
error=error_pred(puntomax)

In [81]:
print(error)

0.00047281323877068556
